In [1]:
###############################################################
# VectorStore, Retriever 준비
###############################################################
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings

COLLECTION_NAME = "olympic_info"
PERSIST_DIRECTORY = "vector_store/chroma/olympic_info"

def get_vectorstore():
    embedding_model = OpenAIEmbeddings(
        model="text-embedding-3-large",
    )

    vector_store = Chroma(
        embedding_function=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )
    return vector_store


def get_retriever(k=10):
    vector_store = get_vectorstore()
    retriever = vector_store.as_retriever(search_kwargs={"k":k})
    return retriever

In [3]:
###########################################################################
# Document 추가
###########################################################################

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

vector_store = get_vectorstore()

path = 'data/olympic.txt'
loader = TextLoader(path, encoding="utf-8")
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50
)

docs = loader.load_and_split(splitter)

ids = vector_store.add_documents(documents=docs)
len(ids)

61

In [4]:
retriever = get_retriever(k=3)
retriever.invoke("IOC에 대해 설명해줘.")

[Document(id='2e4404e0-c605-403f-a71f-2e01715c7897', metadata={'source': 'data/olympic.txt'}, page_content='국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.\n- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아니지만 IOC의 승인을 받은 연맹도 있다.)'),
 Document(id='755c38a7-0f3c-497d-aa82-83de49612453', metadata={'source': 'data/olympic.txt'}, page_content='올림픽은 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 각 올림픽의 위원회(예-벤쿠버동계올림픽조직위원회)로 구성된다. 의사 결정 기구인 IOC는 올림픽 개최 도시를 선정하며, 각 올림픽 대회마다 열리는 올림픽 종목도 IOC에서 결정한다. 올림픽 경기 개최 도시는 경기 축하 의식이 올림픽 헌장에 부합하도록 조직하고 기금을 마련해야 한다. 올림픽 축하 행사로는 여러 의식과 상징을 들 수 있는데 올림픽기나 성화가 그 예이다.'),
 Document(id='41ab152b-562e-47c2-82ca-edfa6a0cb203', metadata={'source': 'data/olympic.txt'}, page_content='- 국가 올림픽 위원회(NOC)는 각국의 올림픽 

# Rerank

## 개념

- RAG의 정확도는 관련 정보의 컨텍스트 내 존재 유무가 아니라 순서가 중요하다. 즉, 관련 정보가 컨텍스트 내 상위권에 위치하고 있을 때 좋은 답변을 얻을 수 있다는 뜻이다. 
- **Rerank**는 RAG 시스템에서 **초기 검색 단계에서 추출된 후보 문서들의 순위를 재조정**하는 기법이다. 
- 벡터 유사도 기반의 빠른 1차 검색 후, 보다 정밀한 모델(예: Cross-encoder, LLM 등)을 활용해 질문과 검색된 문서간의 의미론적 관련성을 평가하여, 실제로 답변 생성에 가장 **적합한 문서들이 상위**에 오도록 순서를 다시 매긴다. 이를 통해 LLM이 더 정확하고 관련성 높은 정보를 바탕으로 답변을 생성할 수 있게 도와준다.

## 방법

- **Cross-encoder 기반 Rerank**  
  - Cross Encoder를 이용해서 순위를 재 지정한다.
  - Cross-encoder
    - 질문과 문서를 같이 입력으로 받아 둘간의 유사도 점수를 예측하도록 학습한 모델.
    - 학습을 두 문장의 유사도록 예측하도록 학습하였기 때문에 단순 유사도 검사 보다 두 문장간의 의미적 관련성등을 이용해 유사도를 예측하기 때문에 더 정확한 결과를 보인다.
  
  - 1차적으로 검색한 문서와 질문간의 유사도를 **cross-encoder**로 다시 계산해서 문서의 순위를 재 조정한다.
    - 
  > - **Bi-encoder**
  >     - 질문 (query)와 문서(document)를 각각 독립적으로 인코딩한 후, 벡터 간 유사도 계산
  >     - Encoder 모델은 개별 문장을 입력받아 embedding vector를 출력한다. 질문과 문서를 각각 encoding한 뒤에 둘 간의 유사도를 계산한다.
  
  ![bi_crosss_encoder](figures/bi_cross_encoder.png)

  \[출처:https://aws.amazon.com/ko/blogs/tech/korean-reranker-rag/\]

- **LLM 기반 Rerank**  
  GPT-3, GPT-4 등 LLM을 활용해 각 문서가 질문에 얼마나 부합하는지 평가하여 순위를 매긴다. 성능이 뛰어나지만 비용이 높고 응답 속도가 느릴 수 있다.
## Rerank RAG 프로세스  
1. 1차 검색(예: 임베딩 기반 벡터 검색)으로 상위 k개 문서 추출.  
2. Reranker 모델에 질문-문서 쌍을 입력.  
3. 각 쌍의 관련성 점수 산출 및 재정렬.  
4. 상위 n개 문서를 LLM의 컨텍스트로 전달하여 답변 생성.

## 장단점

- **장점**  
  - Rerank를 적용하면 단순 벡터 유사도 기반 검색보다 훨씬 정교하게 질문과 관련된 정보를 추출할 수 있다. 
  - 실제로 생성되는 답변의 품질이 크게 향상되며, 도메인 특화 정보나 복잡한 질의에도 높은 정확도를 보인다.

- **단점**  
  - Cross-encoder나 LLM 기반 Rerank는 연산량이 많아 실시간 응답이 필요한 대규모 서비스에선 속도 저하가 발생할 수 있다.
  - 초기 검색 결과(상위 k개)에만 적용하므로, 1차 검색의 품질이 낮으면 Rerank 효과가 제한적일 수 있다.

## CrossEncoder Reranker 예제

In [5]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

In [6]:
# cross encoder (임베딩) 모델이 필요. - cohere 
reranker_model_id = "BAAI/bge-reranker-v2-m3"

retriever = get_retriever()  # 기본 retriever
reranker = HuggingFaceCrossEncoder(model_name=reranker_model_id)
compressor = CrossEncoderReranker(model=reranker, top_n=5) # 압축방식객체 생성 <- 크로스인코더 객체
reranker_retriever = ContextualCompressionRetriever(
    base_retriever=retriever, # 처음 문서를 조회할 retriever
    base_compressor=compressor, # base retriever가 검색한 문서를 압축하는 알고리즘. (Reranker)
)

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/17.2k [00:00<?, ?B/s]

In [9]:
query = "올림픽과 관련된 논란들을 알려주세요."
base_result = retriever.invoke(query)
rerank_result = reranker_retriever.invoke(query)

In [11]:
base_result_str = [d.page_content for d in base_result]
rerank_result_str = [d.page_content for d in rerank_result]

In [12]:
base_result_str

['올림픽',
 '1998년에 몇몇 IOC위원들이 2002년 솔트레이크 시티 동계 올림픽 유치 과정에서 미국에게 미국을 올림픽 개최지로 뽑아달라는 뇌물청탁을 받았다는 것이 폭로되었다. 이에 IOC는 사퇴한 IOC위원 4명과 강제 퇴출된 6명에 대한 조사를 했다. 이 스캔들은 이후에 개최지 선정에서 이와 같은 불미스러운 일이 일어나지 않게 하기 위해서 IOC가 개혁에 착수하도록 하는 긍정적인 역할을 하기도 했다.',
 "BBC 다큐멘터리인 '파노라마'에서는 '매수된 올림픽'이란 주제로 2004년 8월에 방송을 내보내기도 했다. 이때 이 프로그램에서는 2012년 하계 올림픽의 개최지 선정과 관련된 뇌물에 대해서 조사했다. 이 다큐멘터리에서는 특정 후보 도시가 IOC 위원들에게 뇌물수수하는 것이 가능했다고 주장했으며, 특히 파리 시장이었던 베르트랑 들라노에(Bertrand Delanoë)는 영국의 총리인 토니 블레어와 런던올림픽유치위원회가 입후보 규정을 위반했다고 비난했다. 그는 당시 프랑스 대통령이었던 자크 시라크를 목격자로 내세웠지만 시라크 대통령은 이 분쟁에 휘말려드는 것을 주의했으며 인터뷰를 삼갔다. 결국 베르트랑 들라노에의 주장에 대한 조사는 체계적으로 이루어지지는 않았다. 2006년 동계 올림픽을 유치했던 토리노도 이 논쟁에서 빠져나갈 수 없었다. 이번에는 스위스 국적의 IOC위원 마크 호들러(Marc Hodler)가 이 논쟁의 중심이 되었는데, 이 위원은 스위스 시온의 경쟁 도시였던",
 '국제 올림픽 위원회(이하 IOC로 지칭)는 몇몇 위원들이 한 행위에 대해서 비판을 받고 있다. 그 예로 IOC 위원장이었던 에이버리 브런디지와 후안 안토니오 사마란치가 대표적인 사람이다. 브런디지는 20년 넘게 IOC 위원장직을 맡았고 임기 중에 올림픽을 정치적으로 휘말려들지 않게 하기 위해 보호했다. 그러나 그는 남아프리카 공화국 대표단에게 아파르트헤이트와 관련된 이슈를 건드리고 반유대정책을 함으로써 비난을 받았다. 사마란치 위원장 시기 때는 족벌 정치와 부패로 비

In [13]:
rerank_result_str

["BBC 다큐멘터리인 '파노라마'에서는 '매수된 올림픽'이란 주제로 2004년 8월에 방송을 내보내기도 했다. 이때 이 프로그램에서는 2012년 하계 올림픽의 개최지 선정과 관련된 뇌물에 대해서 조사했다. 이 다큐멘터리에서는 특정 후보 도시가 IOC 위원들에게 뇌물수수하는 것이 가능했다고 주장했으며, 특히 파리 시장이었던 베르트랑 들라노에(Bertrand Delanoë)는 영국의 총리인 토니 블레어와 런던올림픽유치위원회가 입후보 규정을 위반했다고 비난했다. 그는 당시 프랑스 대통령이었던 자크 시라크를 목격자로 내세웠지만 시라크 대통령은 이 분쟁에 휘말려드는 것을 주의했으며 인터뷰를 삼갔다. 결국 베르트랑 들라노에의 주장에 대한 조사는 체계적으로 이루어지지는 않았다. 2006년 동계 올림픽을 유치했던 토리노도 이 논쟁에서 빠져나갈 수 없었다. 이번에는 스위스 국적의 IOC위원 마크 호들러(Marc Hodler)가 이 논쟁의 중심이 되었는데, 이 위원은 스위스 시온의 경쟁 도시였던",
 '국제 올림픽 위원회(이하 IOC로 지칭)는 몇몇 위원들이 한 행위에 대해서 비판을 받고 있다. 그 예로 IOC 위원장이었던 에이버리 브런디지와 후안 안토니오 사마란치가 대표적인 사람이다. 브런디지는 20년 넘게 IOC 위원장직을 맡았고 임기 중에 올림픽을 정치적으로 휘말려들지 않게 하기 위해 보호했다. 그러나 그는 남아프리카 공화국 대표단에게 아파르트헤이트와 관련된 이슈를 건드리고 반유대정책을 함으로써 비난을 받았다. 사마란치 위원장 시기 때는 족벌 정치와 부패로 비난받았다. 사마란치가 스페인에서 프랑코 정권에 협력했다는 것도 비판의 이유가 되었다.',
 '현대 올림픽에서는 프로 선수의 참가 불허가 많은 분쟁을 가져왔다. 1912년 하계 올림픽의 근대 5종 경기와 10종 경기에서 우승한 짐 소프는 올림픽에 나가기 전에 준프로야구선수로 활동했다는 게 나중에 밝혀져 메달이 박탈되었다. 소프는 후에 동정적 여론의 힘을 업고 1983년에 메달을 돌려받게 된다. 1936년 동계 올림픽 

# HyDE (Hypothetical Document Embedding)
## 개념
- 질문(query)에 대한 가상의 답변 문서를 생성하고, 이 생성된 가상의 답변 문서를 임베딩하여 검색에 활용하는 기법이다.
- 일반적인 RAG에서 검색은 질문을 임베딩하여 문서 임베딩과 직접 비교한다. 
- 질문("파리는 어떤 도시인가?")과 답변 문서("파리는 프랑스의 수도이며...")는 표현 방식이 다르다는 문제가 있다. 
- 즉 의미적으로는 관련있지만 벡터 공간(임베딩 벡터간의 유사서)에서는 거리가 멀 수가 있다.
- 그래서 HyDE는 질문과 문서가 아니라 질문으로 가상의 답변 문서를 만들고 **가상의 답변문서와 저장된 문서들간의 유사도**를 비교한다.

## HyDE 프로세스

1. 가상 문서 생성
   -  LLM을 사용해 질문에 대한 가상의 답변 문서 생성
   -  이때 성능이 좋은 LLM을 사용하는 것이 좋다.
2. 임베딩 변환
   - `1`에서 생성한 가상 문서를 벡터로 임베딩
3. 유사도 검색
   - 가상 문서 임베딩으로 Vector Store에 저장된 문서들 중 유사한 문서를 검색
4. 답변 생성
   - 검색된 실제 문서를 바탕으로 최종 답변 생성

## 장단점
- **장점**
  - 질문-문서 간 의미적 차이를 해결 해서 정확한 문서 검색 가능
  - 질문 표현 방식에 덜 민감
- **단점**
  - 가상 문서의 품질에 따른 성능 편차가 발생한다.
    - 가상 문서 생성 시 환각(hallucination) 위험
  - 추가적인 LLM 호출로 인한 비용 증가한다.

In [9]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.documents import Document

In [10]:
retriever = get_retriever()
#####################################
# HyDE chain을 구성
#  - 사용자 질문 -> chain -> 가상의 답변문서
#####################################
hyde_mode = ChatOpenAI(model_name="gpt-4.1")                        # 가상답변을 생성하는 모델은 좋은 모델을 사용해야 함
hyde_prompt_template = PromptTemplate(
    template="""
            # Instruction:
            다음 질문에 대해서 완전하고 상세한 답변을 실제 사실에 기반해서 작성해 주세요.
            질문과 관련된 내용만으로 답변을 작성합니다.
            답변과 직접적인 연관성이 없는 내용은 답변에 포함시키지 않습니다.

            # 질문:
            {query}
"""
)
hyde_chain = hyde_prompt_template | hyde_mode | StrOutputParser()

In [12]:
query = "동계 올림픽은 언제부터 시작했나요?"
dummy_answer = hyde_chain.invoke({"query":query})

In [13]:
base_result = retriever.invoke(query)
hyde_result = retriever.invoke(dummy_answer)

In [14]:
[d.page_content for d in base_result]

['동계올림픽\n동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계 올림픽 때 실행하기 불가능한 종목들로 구성되어 있다. 피겨스케이팅, 아이스하키는 각각 1908년과 1920년에 하계올림픽 종목으로 들어가 있었다. IOC는 다른 동계 스포츠로 구성된 새로운 대회를 만들고 싶어 했고, 로잔에서 열린 1921년 올림픽 의회에서 겨울판 올림픽을 열기로 합의했다. 1회 동계올림픽은 1924년, 프랑스의 샤모니에서 11일간 진행되었고, 16개 종목의 경기가 치러졌다. IOC는 동계 올림픽이 4년 주기로 하계 올림픽과 같은 년도에 열리도록 했다. 이 전통은 프랑스의 알베르빌에서 열린 1992년 올림픽 때까지 지속되었으나, 노르웨이의 릴레함메르에서 열린 1994년 올림픽부터 동계 올림픽은 하계 올림픽이 끝난지 2년후에 개최하였다.',
 '올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의',
 '고대올림픽',
 '하계올림픽',
 '열린 년도에 열리게 되었다. 서울에서 열린 1988년 하계 올림픽부터는 하계 올림픽을 개최한 도시는 

In [15]:
[d.page_content for d in hyde_result]

['동계올림픽\n동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계 올림픽 때 실행하기 불가능한 종목들로 구성되어 있다. 피겨스케이팅, 아이스하키는 각각 1908년과 1920년에 하계올림픽 종목으로 들어가 있었다. IOC는 다른 동계 스포츠로 구성된 새로운 대회를 만들고 싶어 했고, 로잔에서 열린 1921년 올림픽 의회에서 겨울판 올림픽을 열기로 합의했다. 1회 동계올림픽은 1924년, 프랑스의 샤모니에서 11일간 진행되었고, 16개 종목의 경기가 치러졌다. IOC는 동계 올림픽이 4년 주기로 하계 올림픽과 같은 년도에 열리도록 했다. 이 전통은 프랑스의 알베르빌에서 열린 1992년 올림픽 때까지 지속되었으나, 노르웨이의 릴레함메르에서 열린 1994년 올림픽부터 동계 올림픽은 하계 올림픽이 끝난지 2년후에 개최하였다.',
 '올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의',
 '국제올림픽위원회(IOC)는 근대 올림픽은 순환 개최로 열리는 세계적인 행사가 되어야 한다고 생각했다. 결국 2회 올림픽은 프랑스 파리에서

In [16]:
# 최종 chain : hyde_chain | 답변처리 chain
# 답변 처리 chain
answer_model = ChatOpenAI(model_name = "gpt-4.1-mini")
template = """# Instruction:
당신은 정확한 정보 제공을 우선시하는 인공지능 어시스턴트입니다.
주어진 Context에 포함된 정보만 사용해서 질문에 답변하세요.
Context에 질문에 대한 명확한 정보가 있는 경우 그 내용을 바탕으로 답변하세요.
Context에 질문에 대한 명확한 정보없을 경우 "정보가 부족해서 답을 알 수 없습니다." 라고 대답합니다.
절대 Context에 없는 내용을 추측하거나 일반 상식을 이용해 답을 만들어서 대답하지 않습니다.

# Context:
{context}

# 질문:
{query}
"""
answer_prompt_template = PromptTemplate(template=template)

answer_chain = answer_prompt_template | answer_model | StrOutputParser()

##########################################
# 최종 chain = hyde_chain + answer_chain
##########################################

def format_docs(docs:list[Document]) -> str:
    return "\n\n".join(doc.page_content for doc in docs)

final_chain = {"context":hyde_chain | retriever | format_docs, "query":lambda x: x["query"]} | answer_chain

In [17]:
query = "올림픽 관련 논란들에 대해 알려주세요."
response = final_chain.invoke({"query":query})

In [18]:
print(response)

올림픽 관련 논란들은 다음과 같습니다.

1. 도핑 문제  
- 1968년 하계 올림픽 근대 5종 경기에서 한스 군나르 리렌바르가 알코올 복용 양성 반응으로 동메달을 박탈당한 첫 사례가 있었고, 1988년 하계 올림픽 육상 100m 금메달리스트 벤 존슨이 스타노졸롤 복용으로 금메달을 박탈당하는 등 도핑 문제가 꾸준히 발생했다.  
- 1999년 세계반도핑기구(WADA)가 설립되었지만 2000년과 2002년 올림픽에서는 약물 양성 반응 선수가 급증했다.  
- 2006년 동계 올림픽에서도 메달리스트 한 명이 도핑으로 메달을 반납했다.  
- 2008년 베이징 올림픽에서는 3,667명 선수가 도핑 검사를 받았으며 올림픽 기간 중 3명만 도핑에 걸렸으나 일부 선수들은 출전금지 조치를 받았다.

2. 프로 선수 참가 논란  
- 처음에는 아마추어 정신을 내세워 프로 선수 참가를 금지했으나, 1912년 하계 올림픽 근대 5종 및 10종 경기에서 짐 소프가 준프로야구 선수 출신으로 메달이 박탈되었고 나중에 메달이 복구되기도 했다.  
- 1936년 동계 올림픽에서는 스위스·오스트리아 스키선수들의 돈벌이 행위가 아마추어 정신 위배로 간주되어 참가를 금지했다.  
- 이후 프로 선수의 참가 허용 문제와 관련된 분쟁이 이어졌다.

3. 정치적 문제 및 전쟁으로 인한 올림픽 취소  
- 제1차 세계대전과 제2차 세계대전으로 인해 1916년, 1940년, 1944년 올림픽이 여러 차례 취소되었다.  
- 2008년 베이징 올림픽 개막식 당일에는 조지아와 러시아 간 남오세티아 전쟁이 발생해 국제 정세 문제와 연계되기도 했다.

4. 올림픽의 상업화 및 부패 의혹  
- 올림픽이 대중매체와 기업 후원으로 상업화되면서 이에 따른 논란이 생겼다.  
- 2004년 BBC 다큐 '파노라마'에서는 2012년 하계 올림픽 개최지 선정과 관련한 뇌물 수수 의혹을 다루었으며, 일부 IOC 위원들이 뇌물을 받았다는 주장이 제기되었다. 조사도 체계적으로 이루어지지 않았다.

5. 기타  
- 도핑 

#  MultiQueryRetriever

## 개념
- 하나의 사용자 질문으로 **여러 개의 다양한 질문을 생성하여 검색**을 수행하는 방법이다.
- 단일 질문의 한계를 극복하고 다각도에서 관련 정보 검색할 수있다.
  - 기본 RAG는 사용자의 질문의 질(quality)에 따라 검색 결과가 좌우된다.
  - 사용자가 한 질문에만 의존하는 것이 아니라 그 질문을 바탕으로 **다양한 의미의 질문들을 생성해서 단일 질문이 가지는 표현의 한계를 보완**한다.
    - 동일한 질문을 다른 각도에서 접근할 수있다.
    - 다양한 어휘와 표현으로 질문을 재구성한다.
- 예)
  - **원본 질문**: "딥러닝의 장점은 무엇인가?"
  - **생성된 질문들**:
    1. "딥러닝이 전통적인 머신러닝보다 나은 점은?"
    2. "딥러닝을 사용하면 얻을 수 있는 이익은?"
    3. "딥러닝의 주요 강점과 특징은?"
    4. "딥러닝 기술의 핵심 우위는?"
## 실행 프로세스

1. 질문 생성   
   - LLM을 사용해 원본 질문을 3-5개의 서로 다른 질문으로 변환
2. 병렬 검색
   - 생성된 각 질문으로 독립적으로 문서 검색 수행
3. 결과 통합
   - 여러 검색 결과를 하나로 병합
4. 중복 제거
   - 동일한 문서가 여러 번 검색된 경우 중복 제거
5. 최종 답변
   - 통합된 문서 세트를 바탕으로 답변 생성

## 장단점
- **장점**
    - 단일 질문으로 놓칠 수 있는 관련 문서 발견 수 있다.
    - 사용자 질문 표현 방식의 한계 극복
    - 더 포괄적이고 완전한 정보 검색 및 수집을 할 수있다.
- **단점**
    - 여러 번의 LLM 호출과 검색 수행이 실행 되므로 **계산비용, 토큰비용, 응답시간이 증가한다.**
    - 생성된 질문의 품질에 따른 성능 편차가 있을 수 있다.
    - 생성된 질문에 때라 원래 질문과 관련성 낮은 문서도 검색될 수 있어 최종 답변을 방해하는 노이즈가 증가할 수있다.

In [20]:
from langchain.retrievers.multi_query import MultiQueryRetriever

model = ChatOpenAI(model_name="gpt-4.1")
retriever = get_retriever()

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever,
    llm=model
    # , prompt=PromptTemplate() -> 질문생성 프롬프트 입력
)
### 실행 순서
# 1. 입력받은 질문을 이용해서 가상의 질문 3개를 생성
# 2. 생성된 질문을 retriever에 전달해서 문서들을 검색
# 3. 중복된 문서들을 제거하고 문서들을 반환

In [25]:
result_docs = multi_query_retriever.invoke("세계 보디빌더 대회인 올림피아 논란에 대해 말해주세요")

In [27]:
print(format_docs(result_docs))

올림픽

국제 올림픽 위원회(이하 IOC로 지칭)는 몇몇 위원들이 한 행위에 대해서 비판을 받고 있다. 그 예로 IOC 위원장이었던 에이버리 브런디지와 후안 안토니오 사마란치가 대표적인 사람이다. 브런디지는 20년 넘게 IOC 위원장직을 맡았고 임기 중에 올림픽을 정치적으로 휘말려들지 않게 하기 위해 보호했다. 그러나 그는 남아프리카 공화국 대표단에게 아파르트헤이트와 관련된 이슈를 건드리고 반유대정책을 함으로써 비난을 받았다. 사마란치 위원장 시기 때는 족벌 정치와 부패로 비난받았다. 사마란치가 스페인에서 프랑코 정권에 협력했다는 것도 비판의 이유가 되었다.

올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의

선수 개인이 자신의 정치적 성향에 대해 표현하기도 했다. 멕시코 시티에서 열린 1968년 하계 올림픽의 육상부문 200m 경기에서 각각 1위와 3위를 한 미국의 토미 스미스와 존 카를로스는 시상식 때 블랙 파워 설루트(Black Power salute , 흑인 차별 반대 행위)를 선보였으며 2위를 한 피터 노먼도 상황을 깨닫고

# MapReduce RAG 방식

## 개요

- RAG(Retrieval-Augmented Generation)에서 검색된 문서들 중 질문과 관련성이 높은 문서만을 선별하여 더 정확한 답변을 생성하는 방법이다.
- 검색된 문서들 중에서 질문 답변에 실제로 도움이 되는 문서만을 LLM을 통해 선별한 후 전달하는 방식이다.

## MapReduce 방식 프로세스
1. Map (문서 검색)
  - 벡터스토어에서 질문과 유사한 문서들을 의미적 유사도 검색으로 찾는다.
  - 이 단계에서는 단순 벡터 유사도만 고려하므로 질문과 직접적인 관련이 없는 문서도 포함될 수 있다.
2. Reduce (문서 선별 및 요약)
   - 검색된 각 문서가 질문 답변에 실제로 도움이 되는지 LLM에게 평가 요청한다.
   - 관련성이 높은 문서들만 선별하여 요약하거나 결합한다.
   - 필요시 여러 문서의 정보를 통합하여 더 응답에 적합한 컨텍스트를 생성한다.
3.  Generate (최종 답변 생성)
    - 질문과 선별된 컨텍스트를 함께 LLM에 전달하여 최종 답변을 생성한다.

## 장단점

- **장점**
  - **높은 정확도**: 질문과 직접 관련된 정보만 사용하여 더 정확한 답변을 생성한다.
  - **노이즈 제거**: 유사하지만 관련 없는 정보로 인한 혼동을 방지한다.
  - **컨텍스트 최적화**: 제한된 토큰 범위 내에서 가장 유용한 정보만 전달한다.
  - **확장성**: 많은 문서가 검색되어도 중요한 정보만 선별하여 처리할 수 있다.
- 단점
  - **추가 비용**: 문서 선별을 위한 LLM 호출로 인한 비용이 증가한다.
  - **처리 시간**: 문서 평가 단계가 추가되어 응답 속도가 저하된다.
  - **복잡성**: 구현과 관리가 더 복잡하다.
  - **의존성**: 문서 선별 성능이 LLM의 판단 능력에 크게 의존한다.

In [43]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, chain
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
retriever = get_retriever(k=5)

In [49]:
# 질문과 문서간의 관련성을 확인하는 chain
reducer_prompt_template = PromptTemplate(
    template="""# Instruction:
    Context의 내용이 질문과 관련성이 있으면 입력된 context를 반환하고 관련성이 없으면 아무것도 출력하지 마세요.
    - 관련성 판단기준
        - Context에 질문에 대한 직접적인 답변이나 답을 위한 단서가 있어야 한다.
        - Context의 정보가 질문을 해결하는데 도움을 줘야한다.

    Context:
        {context}

    질문:
        {query}

    # output indicator:
        - 질문과 context가 관련있으면 context 내용만 정확히 그대로 반환한다. 추가 설명, 내용 수정, 요약 등 변경을 금지한다.,
        - 질문과 context가 관련성이 없으면 아무것도 출력하지 않습니다.,
"""
)
reducer_model = ChatOpenAI(model_name="gpt-4.1-mini")
reducer_chain = reducer_prompt_template | reducer_model | StrOutputParser()

In [41]:
# 테스트 -> 관련이 없는 경우 mapreduce를 통해 빈 문자열 출력
reducer_chain.invoke({"query": "하체운동의 좋은점을 알려줘.", "context": "하체도 좋은 운동입니다."})

'하체도 좋은 운동입니다.'

In [52]:
#####################################################################
# retriever로 문서를 조회 -> reducer_chain에서 질문과의 관련성있는 문서만 추리기

@chain
def map_doc(inputs:dict) -> str:
    """
    Retriever가 조회한 문서들과 query를 받아서 reducer_chain을 이용해 질문과 각 문서간의 관련성을 확인
    관련성있는 문서들만 추려서 반환.

    Args:
        input(dict): 질문과 문서들을 받는다. {"document_list":list[Document], "query":질문}
    Returns:
        str - 질문과 관련있는 문서들을 "\n\n"로 연결해서 반환.
    """
    docs = inputs["document_list"]                  # Retriever가 검색한 문서들
    query = inputs["query"]                         # 사용자 질문
    context = ""                                    # document_list에서 query와 관련된 문서들만 저장할 변수
    for doc in docs:
        res = reducer_chain.invoke({"context":doc.page_content, "query":query})
        if not res.strip():                         # 빈 문자열인 경우 -> 관련 X
            continue
        context += res + "\n\n"

    return context

map_reduce_chain = {"document_list":retriever, "query":RunnablePassthrough()} | map_doc               # RunnablePassthrough를 통해 앞에서 받은 값을 그대로 넘김

In [53]:
map_reduce_chain.invoke("올림픽에 대해 설명해주세요.")

'올림픽\n\n올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의\n\n고대올림픽\n\n감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.\n\n올림픽 경기 종목\n올림픽 경기 종목은 총 33개부문 52개 종목에서 약 400개의 경기로 이루어져있다. 예를 들어서 하계 올림픽 부문인 레슬링은 자유형과 그레코로만형의 두 종목으로 나뉜다. 여기에서 10경기는 남자부, 4

In [59]:
template="""# Instruction:
    Context의 내용이 질문과 관련성이 있으면 입력된 context를 반환하고 관련성이 없으면 아무것도 출력하지 마세요.
    - 관련성 판단기준
        - Context에 질문에 대한 직접적인 답변이나 답을 위한 단서가 있어야 한다.
        - Context의 정보가 질문을 해결하는데 도움을 줘야한다.

    Context:
        {context}

    질문:
        {query}

    # output indicator:
        - 질문과 context가 관련있으면 context 내용만 정확히 그대로 반환한다. 추가 설명, 내용 수정, 요약 등 변경을 금지한다.,
        - 질문과 context가 관련성이 없으면 아무것도 출력하지 않습니다.,
        - 최종 응답 형식은 markdown 형식으로 작성합니다.
"""
final_prompt_template = PromptTemplate(template=template)
final_model = ChatOpenAI(model="gpt-4.1")
# 질문 -> retriever검색 -> map_reduce로 관련문서만 추려내기 -> final_prompt -> final_model -> 최종응답
final_chain = ({"context":map_reduce_chain, "query":RunnablePassthrough()} | final_prompt_template | final_model | StrOutputParser())

In [60]:
query = "올림픽 경기 종목에 대해서 설명해주세요."
response = final_chain.invoke(query)

In [61]:
print(response)

올림픽 경기 종목
올림픽 경기 종목은 총 33개부문 52개 종목에서 약 400개의 경기로 이루어져있다. 예를 들어서 하계 올림픽 부문인 레슬링은 자유형과 그레코로만형의 두 종목으로 나뉜다. 여기에서 10경기는 남자부, 4경기는 여자부로 열리며 분류기준은 체중이다. 하계 올림픽은 26개, 동계 올림픽은 7개 부문으로 이루어져있다. 하계 올림픽에서는 육상, 수영, 펜싱, 체조가 1회 대회때부터 한번도 빠짐없이 정식종목이었으며, 동계 올림픽에서는 크로스컨트리, 피겨 스케이팅, 아이스 하키, 노르딕 복합, 스키 점프, 스피드 스케이팅이 1924년 동계 올림픽부터 빠짐없이 정식종목이었다. 배드민턴, 농구, 배구와 같은 정식종목들은 처음에는 시범종목이었으며 그 후에 정식종목으로 승인 되었다. 야구처럼 예전에는 정식종목 이었지만 지금은 정식 종목에서 빠진 종목도 있다.

2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Programme Commission)를 설립했다. 여기서는 올림픽 종목과 올림픽 종목이 아닌 스포츠를 모두 재검토하는 일을 한다. 이 위원회의 목표는 올림픽 종목에 더 체계적으로 다가가는 것이다. 위원회에서는 우선적으로 올림픽 종목으로 포함되기 위해서는 7개의 기준을 충족시켜야 한다고 말한다. 이 7개의 기준은 역사, 전통, 보편성, 인기도와 잠재성, 선수의 건강, 연맹의 스포츠를 관리할만한 능력, 스포츠를 여는 데에 필요한 비용이다. 예를 들면 2012년 하계 올림픽의 정식종목 후보에 7개 조건을 포함한 비(非)올림픽 스포츠가 올랐고 그 내용은, 골프, 가라테, 럭비, 인라인 스케이팅, 스쿼시였다. 이 스포츠들은 IOC 상임이사회에서 재검토되어 2005년 7월에 열린 싱가포르 총회에서 최종 결정하기로 했다. 결국 5개 중 2개(가라테와 스쿼시) 가 최종 후보로 올라왔으나 가라테와 스쿼시 둘 다

올림픽

2002년에 열린 제114차 IOC 총회에서는 하계 올림픽 종목은 최대 28부문 301개 경기에 10,500명이 참가하는 